In [2]:
import requests
import re
import json


def get_video_cid(bvid):
    """
    功能：通过BV号获取视频的cid（弹幕关联的唯一标识）
    参数：bvid - 视频的BV号（如BV1Yx411v7S2）
    返回：cid字符串或None（失败时）
    """
    # B站视频信息API，传入BV号可获取视频元数据
    api_url = f"https://api.bilibili.com/x/web-interface/view?bvid={bvid}"
    # 模拟浏览器请求头，避免被识别为爬虫
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "Referer": "https://www.bilibili.com/"  # 验证请求来源为B站
    }
    
    try:
        # 发送请求并获取响应
        response = requests.get(api_url, headers=headers, timeout=15)
        # 解析JSON格式的响应数据
        result = response.json()
        
        # 检查API返回状态（B站API成功时code为0）
        if result.get("code") == 0:
            cid = result["data"]["cid"]
            print(f"成功获取cid：{cid}")
            return cid
        else:
            print(f"获取cid失败：{result.get('message', '未知错误')}")
            return None
    except Exception as e:
        print(f"获取cid时发生错误：{str(e)}")
        return None


def crawl_danmaku(cid):
    """
    功能：通过cid爬取弹幕数据并解析
    参数：cid - 视频的cid标识
    返回：解析后的弹幕列表（字典格式）
    """

    danmaku_url = f"https://comment.bilibili.com/{cid}.xml"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "Referer": "https://www.bilibili.com/"
    }
    
    try:
        # 发送请求获取弹幕XML数据
        response = requests.get(danmaku_url, headers=headers, timeout=15)
        response.encoding = "utf-8"  # 强制UTF-8编码，避免中文乱码
        
        if response.status_code != 200:
            print(f"获取弹幕失败，状态码：{response.status_code}")
            return []
        
        # 解析XML中的弹幕数据（每条弹幕格式：<d p="参数">内容</d>）
        danmaku_list = []
        # 正则匹配所有弹幕标签（简化处理，复杂场景建议用lxml库）
        pattern = re.compile(r'<d p="([^"]+)">([^<]+)</d>')
        for params_str, content in pattern.findall(response.text):
            # 解析参数（格式：出现时间(秒),类型,字体大小,颜色,发送时间戳,...）
            params = params_str.split(',')
            if len(params) >= 5:  
                danmaku = {
                    "弹幕内容": content, 
                }
                danmaku_list.append(danmaku)
        
        print(f"成功解析 {len(danmaku_list)} 条弹幕")
        return danmaku_list
    except Exception as e:
        print(f"爬取弹幕时发生错误：{str(e)}")
        return []


def save_danmaku_to_file(danmakus, bvid):
    """
    功能：将弹幕数据保存为JSON文件
    参数：danmakus - 弹幕列表；bvid - 视频BV号（用于文件名）
    """
    filename = f"{bvid}_弹幕数据.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(danmakus, f, ensure_ascii=False, indent=2)
    print(f"弹幕数据已保存至：{filename}")


if __name__ == "__main__":
    target_bvid = "BV16Z4y1K7PM"  
    
    print(f"开始爬取视频 {target_bvid} 的弹幕数据...")
    
    cid = get_video_cid(target_bvid)
    if not cid:
        print("无法获取cid，程序退出")
        exit()
    
    time.sleep(1)
    
    danmakus = crawl_danmaku(cid)
    if not danmakus:
        print("未获取到有效弹幕数据，程序退出")
        exit()
    
    save_danmaku_to_file(danmakus, target_bvid)
    
    print("操作完成")

开始爬取视频 BV16Z4y1K7PM 的弹幕数据...
成功获取cid：225012442
成功解析 1200 条弹幕
弹幕数据已保存至：BV16Z4y1K7PM_弹幕数据.json
操作完成
